In [2]:
from backend import config
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [3]:
# Load the CSV email dataset
data_frame = pd.read_csv(os.path.join(config.DATADIR, 'dataset', 'raw', 'sample_1000_email_dataset.csv'))
data_frame.head(5)

,subject,messages,label
0,Webinar Confirmation: GenAI: The Shift to Data...,"Hi Nilay,\n\nThank you for registering for Gen...",2
1,Invitation: Connor <> Nilay - building RAG job...,"When Wednesday Aug 21, 2024 ⋅ 11am – 11:45am (...",2
2,Thank you for applying to Johns Hopkins APL!,"Dear abc,\n\nThank you for applying to the Joh...",0
3,Garner Health Application Follow Up,"Hi Nilay,\n\nThank you for applying to our Dat...",3
4,Thank you for applying to Perplexity AI,"Hi Nilay,\n\nThank you for applying to the AI ...",3


In [4]:
# Count of each label
data_frame['messages'].groupby(data_frame['label']).count()

label
0    243
1    233
2    257
3    267
Name: messages, dtype: int64

In [5]:
# Download the essential NLTK files for stopwords, punctuations, and lemmatization
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

# Download necessary resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nilay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nilay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nilay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nilay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for removing stopwords, lemmaztize and tokenize the data
def clean_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())  # Convert to lowercase
    # Remove stopwords and punctuation, then lemmatize
    cleaned_tokens = [
        lemmatizer.lemmatize(word) for word in tokens
        if word not in stop_words and word not in string.punctuation
    ]
    return ' '.join(cleaned_tokens)

In [7]:
# Creating a copy of the original data frame for a separate tokanized data frame
data_frame_tokanized = data_frame.copy()
data_frame_tokanized.head(3)

,subject,messages,label
0,Webinar Confirmation: GenAI: The Shift to Data...,"Hi Nilay,\n\nThank you for registering for Gen...",2
1,Invitation: Connor <> Nilay - building RAG job...,"When Wednesday Aug 21, 2024 ⋅ 11am – 11:45am (...",2
2,Thank you for applying to Johns Hopkins APL!,"Dear abc,\n\nThank you for applying to the Joh...",0


In [8]:
# Apply the function to the 'subject' and 'messages' columns and store in 'tokenized_subject' and 'tokenized_messages'
data_frame_tokanized['tokenized_subject'] = data_frame_tokanized['subject'].apply(clean_text)
data_frame_tokanized['tokenized_messages'] = data_frame_tokanized['messages'].apply(clean_text)
data_frame_tokanized = data_frame_tokanized.drop(['subject', 'messages'], axis=1)
data_frame_tokanized.head(5)

,label,tokenized_subject,tokenized_messages
0,2,webinar confirmation genai shift data intellig...,hi nilay thank registering genai shift data in...
1,2,invitation connor nilay building rag job chatb...,wednesday aug 21 2024 ⋅ 11am – 11:45am pacific...
2,0,thank applying john hopkins apl,dear abc thank applying john hopkins applied p...
3,3,garner health application follow,hi nilay thank applying data analyst role ’ re...
4,3,thank applying perplexity ai,hi nilay thank applying ai software engineer r...


In [9]:
# Concatenate the tokenized_subject and tokenized_messages
data_frame_tokanized['email_content'] = data_frame_tokanized['tokenized_subject'] + data_frame_tokanized['tokenized_messages']
data_frame_tokanized.head(3)

,label,tokenized_subject,tokenized_messages,email_content
0,2,webinar confirmation genai shift data intellig...,hi nilay thank registering genai shift data in...,webinar confirmation genai shift data intellig...
1,2,invitation connor nilay building rag job chatb...,wednesday aug 21 2024 ⋅ 11am – 11:45am pacific...,invitation connor nilay building rag job chatb...
2,0,thank applying john hopkins apl,dear abc thank applying john hopkins applied p...,thank applying john hopkins apldear abc thank ...


In [10]:
# Using embed model, convert tokenized frame into embeddings
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
data_frame_tokanized['email_embeddings'] = data_frame_tokanized['email_content'].apply(model_embedding.encode)
data_frame_tokanized.head(3)

,label,tokenized_subject,tokenized_messages,email_content,email_embeddings
0,2,webinar confirmation genai shift data intellig...,hi nilay thank registering genai shift data in...,webinar confirmation genai shift data intellig...,"[-0.068945006, -0.0932267, -0.00037362284, -0...."
1,2,invitation connor nilay building rag job chatb...,wednesday aug 21 2024 ⋅ 11am – 11:45am pacific...,invitation connor nilay building rag job chatb...,"[-0.10992242, 0.008053303, 0.09707958, -0.0057..."
2,0,thank applying john hopkins apl,dear abc thank applying john hopkins applied p...,thank applying john hopkins apldear abc thank ...,"[-0.09812444, 0.028659057, 0.0556533, -0.02935..."


In [28]:
data_frame_embeddings = data_frame_tokanized.copy()
data_frame_embeddings = data_frame_embeddings.drop(['tokenized_subject', 'tokenized_messages', 'email_content'], axis=1)
data_frame_embeddings.head(3)

,label,email_embeddings
0,2,"[-0.068945006, -0.0932267, -0.00037362284, -0...."
1,2,"[-0.10992242, 0.008053303, 0.09707958, -0.0057..."
2,0,"[-0.09812444, 0.028659057, 0.0556533, -0.02935..."


In [12]:
# Install required libraries for model fine-tuning
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

In [29]:
# Convert the embeddings to tensor format
data_frame_embeddings['email_embeddings'] = data_frame_embeddings['email_embeddings'].apply(lambda x: torch.tensor(x))
data_frame_embeddings.head(1)

,label,email_embeddings
0,2,"[tensor(-0.0689), tensor(-0.0932), tensor(-0.0..."


In [30]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(data_frame_embeddings, test_size=0.2, random_state=42)

In [31]:
# Custom Dataset class to load the data into the format required by transformers
class EmailDataset(Dataset):
    def __init__(self, df, device=None):
        self.input_ids = df['email_embeddings'].tolist()
        self.labels = df['label'].tolist()
        self.device = device

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_ids_tensor = torch.tensor(self.input_ids[idx], dtype=torch.long)
        labels_tensor = torch.tensor(self.labels[idx], dtype=torch.long)
        if self.device:
            input_ids_tensor = input_ids_tensor.to(self.device)
            labels_tensor = labels_tensor.to(self.device)
        return {'input_ids': input_ids_tensor, 'labels': labels_tensor}

# Load the dataset
train_dataset = EmailDataset(train_df, device='cpu')  # Pass device as needed
val_dataset = EmailDataset(val_df, device='cpu')

In [16]:
# Load the DistilBertForSequenceClassification model 
from transformers import DistilBertTokenizer

# Load the tokenizer and model
tokenizer_DBSC = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_DBSC = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Defining the training arguments for fine-tuning of the DBSC model.
training_args = TrainingArguments(
    output_dir=os.path.join(config.DATADIR, 'ml_models'),          # Output directory for saving model
    num_train_epochs=5,              # Number of training epochs
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=16,   # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps
    weight_decay=0.01,               # Strength of weight decay
    logging_dir=os.path.join(config.DATADIR, 'ml_models', 'logs'),            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluation strategy
    save_strategy="epoch",           # Save strategy
    load_best_model_at_end=True,     # Load the best model when finished training
)

c:\Users\nilay\OneDrive - Cal State Fullerton (1)\Desktop\NILAY-TO-JOB-DATA\SPRING 2024\Projects\Gmail-JobEmail-Classifier\myvenv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
from transformers import Trainer, TrainingArguments

trainer = Trainer(
    model=model_DBSC,                         # The model to train
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset,            # Validation dataset
    tokenizer=tokenizer_DBSC                  # Tokenizer
)

C:\Users\nilay\AppData\Local\Temp\ipykernel_5560\630175910.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
# Fine-tune the model
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]C:\Users\nilay\AppData\Local\Temp\ipykernel_5560\970735130.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor = torch.tensor(self.input_ids[idx], dtype=torch.long)
                                                  
  2%|▏         | 10/500 [03:32<2:30:48, 18.47s/it]

{'loss': 1.3914, 'grad_norm': 3.272550582885742, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.1}


                                                  
  4%|▍         | 20/500 [06:11<2:07:54, 15.99s/it]

{'loss': 1.3895, 'grad_norm': 2.399317502975464, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.2}


                                                  
  6%|▌         | 30/500 [08:49<2:03:25, 15.76s/it]

...

{'eval_loss': 1.3923410177230835, 'eval_runtime': 114.085, 'eval_samples_per_second': 1.753, 'eval_steps_per_second': 0.114, 'epoch': 5.0}


                                                   
100%|██████████| 500/500 [2:23:52<00:00, 15.19s/it]

{'train_runtime': 8633.0258, 'train_samples_per_second': 0.463, 'train_steps_per_second': 0.058, 'train_loss': 1.3922538299560547, 'epoch': 5.0}


100%|██████████| 500/500 [2:23:55<00:00, 17.27s/it]


TrainOutput(global_step=500, training_loss=1.3922538299560547, metrics={'train_runtime': 8633.0258, 'train_samples_per_second': 0.463, 'train_steps_per_second': 0.058, 'total_flos': 397416370176000.0, 'train_loss': 1.3922538299560547, 'epoch': 5.0})

In [35]:
# Evaluate the model
trainer.evaluate()

C:\Users\nilay\AppData\Local\Temp\ipykernel_5560\970735130.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_tensor = torch.tensor(self.input_ids[idx], dtype=torch.long)
100%|██████████| 13/13 [01:51<00:00,  8.57s/it]


{'eval_loss': 1.391973614692688,
 'eval_runtime': 119.5655,
 'eval_samples_per_second': 1.673,
 'eval_steps_per_second': 0.109,
 'epoch': 5.0}

In [ ]:
# The email message to classify
email_message = """

"""

# Tokenize and encode the message
inputs = tokenizer_DBSC(email_message, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Make sure the model is in evaluation mode
model_DBSC.eval()

# Perform prediction
with torch.no_grad():
    outputs = model_DBSC(**inputs)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()

print(f"Predicted label: {predicted_label}")

Predicted label: 3
